<a href="https://colab.research.google.com/github/leiyinghao/colab-demo/blob/main/redis_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

注意：这个笔记是根据[redis-stack](https://redis.io/docs/latest/develop/get-started/vector-database/) 完成的；需要自己从[redislabs](https://app.redislabs.com) 获取对应的 stack 实例账号密码完成实验；

In [ ]:
!pip --version
!python --version


pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Python 3.10.12


In [ ]:
!pip install redisearch
!pip install sentence-transformers
!pip install numpy
!pip install pandas
!pip install redis
!pip install requests

In [ ]:
# 安装 redis-py
!pip install redis

# 如果使用 redisearch-py
!pip install redisearch


In [ ]:
import json
import time

import numpy as np
import pandas as pd
import redis
import requests
# 导入了Redis搜索模块中的四个字段类型：NumericField、TagField、TextField和VectorField。这些字段类型可以用于定义搜索索引的结构和类型，以便在Redis中进行高效的数据检索操作。
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

# import 包需要安装对应的依赖
# ```
#   pip install redisearch
#   pip install sentence-transformers
# ```
print("package import done")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


package import done


In [ ]:

url = "https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/main/data/bikes.json"
response = requests.get(url)
bikes = response.json()
print(len(bikes))
json.dumps(bikes[0], indent=2)
print("---------json first data is ")
print(json.dumps(bikes[0], indent=2))

#todo here replace your host port and password
client = redis.Redis(
  host='redis-19791.c299.asia-northeast1-1.gce.redns.redis-cloud.com',
  port=19791,
  password='',
  decode_responses=True)

#client = redis.Redis(host="localhost", port=16379, decode_responses=True)

res = client.ping()
print(res)
# >>> True

11
---------json first data is 
{
  "model": "Jigger",
  "brand": "Velorim",
  "price": 270,
  "type": "Kids bikes",
  "specs": {
    "material": "aluminium",
    "weight": "10"
  },
  "description": "Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids\u2019 pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin\u2019 little bike is not ideal for a nervous first-time rider, but it\u2019s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one\u2019s need for speed. It\u2019s a single speed bike that makes learning to pump pedals simple and intuitive. It even has  a handle in the bottom of the saddle so you can easily help your child during training!  The Jigger is among the most lightweight children\u2019s bikes on the planet. It is designed so that 2-3 y

In [ ]:
pipeline = client.pipeline()
for i, bike in enumerate(bikes, start=1):
    redis_key = f"bikes:{i:03}"
    pipeline.json().set(redis_key, "$", bike)
res = pipeline.execute()
# >>> [True, True, True, True, True, True, True, True, True, True, True]

res = client.json().get("bikes:010", "$.model")
# >>> ['Summit']
print(res)

['Summit']


In [ ]:

keys = sorted(client.keys("bikes:*"))
# >>> ['bikes:001', 'bikes:002', ..., 'bikes:011']
keys

['bikes:001',
 'bikes:002',
 'bikes:003',
 'bikes:004',
 'bikes:005',
 'bikes:006',
 'bikes:007',
 'bikes:008',
 'bikes:009',
 'bikes:010',
 'bikes:011']

In [ ]:


descriptions = client.json().mget(keys, "$.description")
descriptions = [item for sublist in descriptions for item in sublist]
descriptions

['Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids’ pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin’ little bike is not ideal for a nervous first-time rider, but it’s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one’s need for speed. It’s a single speed bike that makes learning to pump pedals simple and intuitive. It even has  a handle in the bottom of the saddle so you can easily help your child during training!  The Jigger is among the most lightweight children’s bikes on the planet. It is designed so that 2-3 year-olds fit comfortably in a molded ride position that allows for efficient riding, balanced handling and agility. The Jigger’s frame design and gears work together so your buddingbiker can stand up out of the seat, stop rapidly

In [ ]:

# 创建了一个名为embedder的SentenceTransformer对象，该对象使用"msmarco-distilbert-base-v4"预训练模型。
# 然后，使用embedder对象的encode方法将descriptions列表中的每个句子转换为向量表示，
# 并将这些向量表示转换为float32类型，并将其存储为一个列表。
# 最后，通过获取列表中第一个向量的长度，确定了向量的维度，并将其存储在VECTOR_DIMENSION变量中。
# from sentence_transformers import SentenceTransformer
print(" will load model ... ")
embedder = SentenceTransformer("msmarco-distilbert-base-v4")
print("load model msmarco ... ")
embeddings = embedder.encode(descriptions).astype(np.float32).tolist()
print("len is : " + str(len(embeddings)))
VECTOR_DIMENSION = len(embeddings[0])
VECTOR_DIMENSION
# >>> 768

 will load model ... 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

load model msmarco ... 
len is : 11


768

In [ ]:

print("VECTOR_DIMENSION is " + str(VECTOR_DIMENSION))

pipeline = client.pipeline()
for key, embedding in zip(keys, embeddings):
    print("key is "+key)
    pipeline.json().set(key, "$.description_embeddings", embedding)
pipeline.execute()
print("pipe done ... ")
# >>> [True, True, True, True, True, True, True, True, True, True, True]

VECTOR_DIMENSION is 768
key is bikes:001
key is bikes:002
key is bikes:003
key is bikes:004
key is bikes:005
key is bikes:006
key is bikes:007
key is bikes:008
key is bikes:009
key is bikes:010
key is bikes:011
pipe done ... 


# 新段落

In [ ]:
res = client.json().get("bikes:010")
# >>>
# {
#   "model": "Summit",
#   "brand": "nHill",
#   "price": 1200,
#   "type": "Mountain Bike",
#   "specs": {
#     "material": "alloy",
#     "weight": "11.3"
#   },
#   "description": "This budget mountain bike from nHill performs well..."
#   "description_embeddings": [
#     -0.538114607334137,
#     -0.49465855956077576,
#     -0.025176964700222015,
#     ...
#   ]
# }

# 这个schema可以用于将JSON数据转换为特定格式，以便进行文本检索或向量检索等操作。
schema = (
    TextField("$.model", no_stem=True, as_name="model"),
    TextField("$.brand", no_stem=True, as_name="brand"),
    NumericField("$.price", as_name="price"),
    TagField("$.type", as_name="type"),
    TextField("$.description", as_name="description"),
    VectorField(
        "$.description_embeddings",
        "FLAT",
        {
            "TYPE": "FLOAT32",
            "DIM": VECTOR_DIMENSION,
            "DISTANCE_METRIC": "COSINE",
        },
        as_name="vector",
    ),
)
# 创建了一个IndexDefinition对象definition，指定了索引的前缀bikes:和类型为JSON。
definition = IndexDefinition(prefix=["bikes:"], index_type=IndexType.JSON)
res = client.ft("idx:bikes_vss").create_index(
    fields=schema, definition=definition
)
# >>> 'OK'

In [ ]:
schema


(<redis.commands.search.field.TextField at 0x7dca2052eb90>,
 <redis.commands.search.field.VectorField at 0x7dc8d5e0f790>)

In [ ]:

info = client.ft("idx:bikes_vss").info()
num_docs = info["num_docs"]
indexing_failures = info["hash_indexing_failures"]
# print(f"{num_docs} documents indexed with {indexing_failures} failures")
# >>> 11 documents indexed with 0 failures

In [ ]:

query = Query("@brand:Peaknetic")
res = client.ft("idx:bikes_vss").search(query).docs
# print(res)
# >>> [Document {'id': 'bikes:008', 'payload': None, 'brand': 'Peaknetic', 'model': 'Soothe Electric bike', 'price': '1950', 'description_embeddings': ...

query = Query("@brand:Peaknetic").return_fields("id",
                                                "brand", "model", "price")
res = client.ft("idx:bikes_vss").search(query).docs
res

[Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'},
 Document {'id': 'bikes:008', 'payload': None, 'brand': 'Peaknetic', 'model': 'Soothe Electric bike', 'price': '1950'}]

In [ ]:
# print(res)
# >>> [Document {'id': 'bikes:008', 'payload': None, 'brand': 'Peaknetic', 'model': 'Soothe Electric bike', 'price': '1950'}, Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'}]

query = Query("@brand:Peaknetic @price:[0 1000]").return_fields(
    "id", "brand", "model", "price"
)
res = client.ft("idx:bikes_vss").search(query).docs
print(res)
# >>> [Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'}]


[Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'}]


In [ ]:

queries = [
    "Bike for small kids",
    "Best Mountain bikes for kids",
    "Cheap Mountain bike for kids",
    "Female specific mountain bike",
    "Road bike for beginners",
    "Commuter bike for people over 60",
    "Comfortable commuter bike",
    "Good bike for college students",
    "Mountain bike for beginners",
    "Vintage bike",
    "Comfortable city bike",
]

encoded_queries = embedder.encode(queries)
len(encoded_queries)
# >>> 11

11

In [ ]:
def create_query_table(query, queries, encoded_queries, extra_params={}):
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = (
            client.ft("idx:bikes_vss")
            .search(
                query,
                {
                    "query_vector": np.array(
                        encoded_query, dtype=np.float32
                    ).tobytes()
                }
                | extra_params,
            )
            .docs
        )

        #遍历查找的结果 组装为新的格式，放入results_list
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append(
                {
                    "query": queries[i],
                    "score": vector_score,
                    "id": doc.id,
                    "brand": doc.brand,
                    "model": doc.model,
                    "description": doc.description,
                }
            )

    # Optional: convert the table to Markdown using Pandas
    queries_table = pd.DataFrame(results_list)
    queries_table.sort_values(
        by=["query", "score"], ascending=[True, False], inplace=True
    )
    queries_table["query"] = queries_table.groupby("query")["query"].transform(
        lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
    )
    queries_table["description"] = queries_table["description"].apply(
        lambda x: (x[:497] + "...") if len(x) > 500 else x
    )
    mddoc = queries_table.to_markdown(index=False)
    #在这里输出 markdonw 的结果，不然什么信息也不会在控制台打印输出
    print(mddoc)

#第一个查询语句（query）使用了Query类来构建。它执行了一个KNN（最近邻）算法，其中查询向量是$query_vector。查询结果将根据vector_score进行排序，并返回vector_score、id、brand、model和description字段。该查询使用了方言2。
# KNN 获取的是近邻
#   .dialect(2) 方言2 docs https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/dialects/
query = (
    Query("(*)=>[KNN 3 @vector $query_vector AS vector_score]")
    .sort_by("vector_score")
    .return_fields("vector_score", "id", "brand", "model", "description")
    .dialect(2)
)
print(query)
create_query_table(query, queries, encoded_queries)
# >>> | Best Mountain bikes for kids     |    0.54 | bikes:003... (+ 32 more results)
#这里输出结果是有点类似问答，queries[i] 对应 3 个物品


| query                            |   score | id        | brand      | model                | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:---------------------------------|--------:|:----------|:-----------|:---------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#第二个查询语句（hybrid_query）与第一个类似，但它在查询条件中添加了一个品牌过滤器（@brand:Peaknetic）。同样，查询结果将根据vector_score进行排序，并返回相同的字段。该查询也使用了方言2。
hybrid_query = (
    Query("(@brand:Peaknetic)=>[KNN 3 @vector $query_vector AS vector_score]")
    .sort_by("vector_score")
    .return_fields("vector_score", "id", "brand", "model", "description")
    .dialect(2)
)
create_query_table(hybrid_query, queries, encoded_queries)

# >>> | Best Mountain bikes for kids     |    0.3  | bikes:008... (+22 more results)




| query                            |   score | id        | brand     | model                | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:---------------------------------|--------:|:----------|:----------|:---------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
range_query = (
    Query(
        "@vector:[VECTOR_RANGE $range $query_vector]=>{$YIELD_DISTANCE_AS: vector_score}"
    )
    .sort_by("vector_score")
    .return_fields("vector_score", "id", "brand", "model", "description")
    .paging(0, 4)
    .dialect(2)
)
print("queries[:1] is :",queries[:2])

create_query_table(
    range_query, queries[:2], encoded_queries[:2], {"range": 0.55}
)
# >>> | Bike for small kids |    0.52 | bikes:001 | Velorim    |... (+1 more result)

queries[:1] is : ['Bike for small kids', 'Best Mountain bikes for kids']
| query                        |   score | id        | brand      | model       | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:-----------------------------|--------:|:----------|:-----------|:------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------